# Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np

# Load data
images = np.load('/content/drive/MyDrive/Datasets/UDIAT/Fused/images.npy')
labels = np.load('/content/drive/MyDrive/Datasets/UDIAT/Fused/labels.npy')
classes = ["Benign", "Malignant"]

In [ ]:
from sklearn.model_selection import train_test_split

# Define the ratios for splitting
train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.2

# Split the data into training, validation, and test sets
x_train, x_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=1 - train_ratio, random_state=42)

x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=test_ratio / (test_ratio + val_ratio), random_state=42)

In [ ]:
# Print the number of samples in each set
print(f"Number of samples in training set: {len(x_train)}")
print(f"Number of samples in validation set: {len(x_val)}")
print(f"Number of samples in test set: {len(x_test)}")

In [ ]:
# Print the number of samples in each class within each set
for set_name, x_set, y_set in [("Training", x_train, y_train), ("Validation", x_val, y_val), ("Test", x_test, y_test)]:
    print(f"Samples in {set_name} set:")
    for class_label, class_name in enumerate(classes):
        num_samples = np.sum(y_set == class_label)
        print(f"  Class '{class_name}': {num_samples}")


In [ ]:
from sklearn.preprocessing import LabelBinarizer
# Use LabelBinarizer for one-hot encoding
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_val = label_binarizer.fit_transform(y_val)
y_test = label_binarizer.fit_transform(y_test)

In [ ]:
# Print the number of samples in each set
print(f"Shape of labels in training set: {np.shape(y_train)}")
print(f"Shape of labels in validation set: {np.shape(y_val)}")
print(f"Shape of labels in test set: {np.shape(y_test)}")

## Plot few random samples

In [ ]:
# Define a function to plot random samples of benign and malignant images
import matplotlib.pyplot as plt
def plot_random_samples(images, labels, class_name, num_samples=3):
    # Get indices of images with the specified class
    class_indices = np.where(labels == class_name)[0]

    # Randomly select num_samples indices
    random_indices = np.random.choice(class_indices, num_samples, replace=False)

    # Create a subplots grid
    fig, axes = plt.subplots(1, num_samples, figsize=(6, 4))

    for i, idx in enumerate(random_indices):
        ax = axes[i]
        ax.imshow(images[idx])
        ax.set_title(f'Sample {i+1}')
        ax.axis('off')

    plt.show()

In [ ]:
# Plot 3 random samples of benign images from training data
plot_random_samples(images, labels, class_name=0, num_samples=3)

In [ ]:
# Plot 3 random samples of malignant images from training data
plot_random_samples(images, labels, class_name=1, num_samples=3)

# Utility Functions

In [ ]:
# Load Libraries
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Initialize the multilayer perceptron classifier
mlp = MLPClassifier()

# Initialize the k-nearest neighbors classifier
knn = KNeighborsClassifier()

# Initialize the random forest classifier
rf = RandomForestClassifier()

# Initialize the gradient boosting classifier
gb = GradientBoostingClassifier()

# Create a voting classifier with weights for the individual classifiers
clf = VotingClassifier(
    estimators=[('mlp', mlp), ('knn', knn), ('rf', rf), ('gb', gb)],
    weights=[1, 1, 2, 1], voting='soft')


classifiers = ['RandomForest',
               'AdaBoost',  'DecisionTree',
               'KNeighbors','MLP','GradientBoosting', 'Proposed']
models = [RandomForestClassifier(n_estimators=200, random_state=0),
          AdaBoostClassifier(random_state = 0),
          DecisionTreeClassifier(random_state=0),
          KNeighborsClassifier(),
          MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1),
          GradientBoostingClassifier(random_state=0),
          clf]

In [ ]:
# Define sensitivity and specificity functions
def sensitivity(y_true, y_pred):
    tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred).ravel()
    return tp / (tp + fn)

def specificity(y_true, y_pred):
    tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Score of classifiers
def acc_score(X_train,X_test,Y_train,y_true):
    Score = pd.DataFrame({"Classifier":classifiers})

    acc = []
    pr = []
    re = []
    f1 = []
    auc = []
    spec = []
    sen = []

    for i in models:
        model = i
        model.fit(X_train,Y_train)
        y_pred = model.predict(X_test)

        acc.append(metrics.accuracy_score(y_true, y_pred))
        pr.append(metrics.precision_score(y_true, y_pred))
        re.append(metrics.recall_score(y_true, y_pred))
        f1.append(metrics.f1_score(y_true, y_pred))
        area = metrics.roc_auc_score(y_true, model.predict_proba(X_test)[:, 1])
        auc.append(area)
        sen.append(sensitivity(y_true, y_pred))
        spec.append(specificity(y_true, y_pred))

    Score["Accuracy"] = acc
    Score["Precision"] = pr
    Score["Recall"] = re
    Score["Sensitivity"] = sen
    Score["Specificity"] = spec
    Score["F1_Score"] = f1
    Score["ROC_AUC"] = auc

    Score.sort_values(by="F1_Score", ascending=False,inplace = True)
    Score.reset_index(drop=True, inplace=True)
    return Score


# Ablation Study

## Classification with CNN Model

### Build and Compile Model

In [ ]:
from keras.applications import MobileNet
import tensorflow as tf

# MobileNet Model
def create_model(SIZE):
  model =  MobileNet(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
  stringlist = []
  model.summary(print_fn=lambda x: stringlist.append(x))
  short_model_summary = "\n".join(stringlist)
  print(short_model_summary)
  for layer in model.layers:
    layer.trainable = False
  # Flatten the output layer to 1 dimension
  x = tf.keras.layers.Flatten()(model.output)

  # Add a fully connected layer with 512 hidden units and ReLU activation
  x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)

  # Add a dropout rate of 0.3
  x = tf.keras.layers.Dropout(0.3)(x)

  # Add a batch normalization layer
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)

  # Add a dropout rate of 0.2
  x = tf.keras.layers.Dropout(0.2)(x)

  # Add a batch normalization layer
  x = tf.keras.layers.BatchNormalization()(x)

  # Add a final sigmoid layer for classification
  output = tf.keras.layers.Dense(1 , activation='sigmoid')(x)

  model = tf.keras.Model(model.input, output)

  return model

In [ ]:
# Plot training history
import matplotlib.pyplot as plt
def plot_history(history=None):
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    epochsn = np.arange(1, len(train_loss)+1)
    plt.figure(figsize = (20,7))

    plt.subplot(1,3,1)
    plt.plot(epochsn,train_loss, 'b', label='Training Loss')
    plt.plot(epochsn,val_loss, 'r', label='Validation Loss')
    plt.grid(color='gray', linestyle='--')
    plt.legend()
    plt.title('Training Loss vs Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

    plt.subplot(1,3,2)
    plt.plot(epochsn, train_acc, 'b', label='Training Accuracy')
    plt.plot(epochsn, val_acc, 'r', label='Validation Accuracy')
    plt.grid(color='gray', linestyle='--')
    plt.legend()
    plt.title('Training Accuracy vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.savefig("Learning_Curves.png", dpi=300)
    plt.show()

In [ ]:
# Create the final model
SIZE = 224
model = create_model(SIZE)

In [ ]:
# Add early stopping and reduce learning rate callbacks
mc = tf.keras.callbacks.ModelCheckpoint("trained_model.h5", monitor='val_accuracy', mode='max',verbose=1, save_best_only=True)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, restore_best_weights=True, patience=20)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

In [ ]:
# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
stringlist = []
model.summary(print_fn=lambda x: stringlist.append(x))
short_model_summary = "\n".join(stringlist)
print(short_model_summary)

### Train Model

In [ ]:
# Apply some augmentation to control overfitting
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(images)

train_generator = datagen.flow(x=x_train, y=y_train, batch_size=32)

In [ ]:
# Train the Model
from datetime import datetime
start = datetime.now()
model_history = model.fit(train_generator, steps_per_epoch=len(x_train) // 32, batch_size=32, epochs=200, validation_data=(x_val, y_val), callbacks=[es, mc, reduce_lr])
stop = datetime.now()

In [ ]:
# Print the Training Time

training_time = stop - start
print('Model training time is :', training_time)

In [ ]:
# Plot the history
plot_history(model_history)

### Evaluate Model on Test data

In [ ]:
# Make predictions on the test set
model = tf.keras.models.load_model('/content/trained_model.h5')
y_pred = model.predict(x_test)

In [ ]:
# Calculate scores
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
accuracy = accuracy_score(y_test, y_pred.round())
sensitivity = recall_score(y_test, y_pred.round())
specificity = recall_score(y_test, y_pred.round(), pos_label=0)
f1 = f1_score(y_test, y_pred.round())
roc = roc_auc_score(y_test, y_pred)

In [ ]:
# Print the results
print("scores")
print("==================================================")
print("Accuracy score: %.4f" % (accuracy))
print("Sensitivity score: %.4f" % (sensitivity))
print("Specificity score: %.4f" % (specificity))
print("F1 score: %.4f" % (f1))
print("roc_auc score: %.4f" % (roc))
print("==================================================")

In [ ]:
# Plot Confusion Matrix
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(y_true, y_pred):

    # Create a confusion matrix
    y_pred_binary = y_pred.round()

    cm = confusion_matrix(y_true, y_pred_binary)

    # Plot the confusion matrix as a heatmap
    plt.imshow(cm, cmap=plt.cm.Blues)
    plt.ylabel('True label', fontsize=14)
    plt.xlabel('Predicted label', fontsize=14)
    plt.xticks([0, 1], ['Benign', 'Malignant'], fontsize=14)
    plt.yticks([0, 1], ['Benign', 'Malignant'], fontsize=14)
    plt.colorbar()

    # Add the values inside the cells
    for i in range(2):
        for j in range(2):
            plt.text(j, i, cm[i, j],
                     fontsize=20,  # specify the desired font size
                     horizontalalignment='center',
                     color='black')
    # Show the plot
    plt.savefig("Confusion_Matrix.png", dpi=300)
    plt.show()
plot_confusion_matrix(y_test, y_pred)

In [ ]:
# Plot ROC Curve
from sklearn.metrics import roc_curve, auc
def plot_roc_curve(y_true, y_pred):
    # Calculate the false positive rate and true positive rate for the ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    # Plot the ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([-0.01, 1.0])
    plt.ylim([0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title('Receiver operating characteristic', fontsize=14)
    plt.legend(loc="lower right", fontsize=14)
    plt.savefig("ROC_Curve.png", dpi=300)
    plt.show()

plot_roc_curve(y_test, y_pred)

## Feature Extraction

In [ ]:
# Load the pre-trained model
model = tf.keras.models.load_model('/content/trained_model.h5')

# Get the total number of layers in the model
total_layers = len(model.layers)
print("Total Layers in the Model are: ", total_layers)

# Define the index of the layer for feature extraction (second-to-last layer in this case)
feature_extraction_layer_index = total_layers - 7

# Get the name of the feature extraction layer
feature_extraction_layer_name = model.layers[feature_extraction_layer_index].name
print("Name of the Feature Extraction Layer: ", feature_extraction_layer_name)


# Specify the layers from which you want to extract features
feature_extraction_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer(feature_extraction_layer_name).output)

print("Extracting features from images...................")
# Extract features from images
features = feature_extraction_model.predict(images)

In [ ]:
# Check the shape of the feature arrays
print("Shape of features:", features.shape)

In [ ]:
# Save the feature arrays as numpy arrays
np.save('features.npy', features)

### Visualize Using t-SNE

Apply t-SNE to reduce the dimensionality of your extracted features.

In [ ]:
from sklearn.manifold import TSNE

# Create a t-SNE model with the desired number of dimensions (e.g., 2 for 2D visualization)
tsne = TSNE(n_components=2, perplexity=30, random_state=42)

# Fit the t-SNE model to the features and transform them to the lower-dimensional space
reduced_features = tsne.fit_transform(features)


Adjust the perplexity parameter to control the balance between preserving global and local structure in the visualization. You may need to experiment with different values to find the most suitable one for your data.

In [ ]:
# Assuming test_labels contains the true labels (0 for Benign, 1 for Malignant)
plt.figure(figsize=(6, 4))
plt.scatter(reduced_features[labels == 0, 0], reduced_features[labels == 0, 1], label='Benign', alpha=0.5)
plt.scatter(reduced_features[labels == 1, 0], reduced_features[labels == 1, 1], label='Malignant', alpha=0.5)
plt.scatter(reduced_features[labels == 2, 0], reduced_features[labels == 2, 1], label='Normal', alpha=0.5)
plt.legend()
plt.title('t-SNE Visualization of Extracted Features')
#plt.xlabel('t-SNE Component 1')
#plt.ylabel('t-SNE Component 2')
plt.axis("off")
plt.savefig("tSNE_Visual.png", dpi=300)
plt.show()

## Classification with Ensemble Model after Feature Extraction using CNN

In [ ]:
import numpy as np

# Load Training data
features = np.load('/content/features.npy')
labels = np.load('/content/drive/MyDrive/Datasets/UDIAT/Fused/labels.npy')
classes = ["Benign", "Malignant"]

from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=42)

# Print the number of samples in each set
print(f"Shape of training set: {np.shape(train_features)}")
print(f"Shape of test set: {np.shape(test_features)}")
print(f"Shape of labels in training set: {np.shape(train_labels)}")
print(f"Shape of labels in test set: {np.shape(test_labels)}")

# Print the number of samples in each class within each set
for set_name, x_set, y_set in [("Training", train_features, train_labels), ("Test", test_features, test_labels)]:
    print(f"Samples in {set_name} set:")
    for class_label, class_name in enumerate(classes):
        num_samples = np.sum(y_set == class_label)
        print(f"  Class '{class_name}': {num_samples}")

############################################################################

In [ ]:
# Evaluate
scores = acc_score(train_features, test_features, train_labels, test_labels)

In [ ]:
# Sorting by column 'Classifier'
sorted_scores = scores.sort_values(by=['F1_Score'], ascending=False)
print("Results on test data")
print("-----------------------------")
sorted_scores.head(10)

In [ ]:
sorted_scores.to_csv("sorted_score_feature_results_UDIAT.csv")

## Classification with Ensemble Model after Feature Extraction using CNN and Applying GA

In [ ]:
## Genetic ALgorithm Functions

import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt
from tqdm import tqdm
# Constants
POPULATION_SIZE = 30
MAX_GENERATIONS = 50

def initialize_population(num_features, population_size):
    population = []
    for _ in range(population_size):
        solution = random.sample(range(num_features), random.randint(1, num_features))
        population.append(solution)
    return population


def roulette_wheel_selection(population, fitnesses):
    # Normalize fitness values
    total_fitness = sum(fitnesses)
    normalized_fitnesses = [f/total_fitness for f in fitnesses]

    # Generate roulette wheel
    wheel = []
    current_position = 0
    for f in normalized_fitnesses:
        current_position += f
        wheel.append(current_position)

    # Select parents
    parents = []
    for i in range(len(population)):
        r = random.random()
        for j, w in enumerate(wheel):
            if r <= w:
                parents.append(population[j])
                break

    return parents


def two_point_crossover(parent1, parent2):
    # Choose crossover points
    point1 = random.randint(0, len(parent1))
    point2 = random.randint(point1, len(parent1))

    # Create offspring
    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]

    return child1, child2



def mutate(solution, num_features, mutation_rate):
    # Determine whether to add or remove a feature
    if random.random() < mutation_rate:
        # Add a random feature
        solution.append(random.randint(0, num_features-1))
    elif len(solution) > 1:
        # Remove a random feature
        solution.remove(random.choice(solution))
    return solution



def select_fittest(population, fitnesses):
    # Sort population and fitnesses by fitness values
    population_fitnesses = list(zip(population, fitnesses))
    population_fitnesses.sort(key=lambda x: x[1], reverse=True)
    sorted_population, sorted_fitnesses = zip(*population_fitnesses)

    # # Return sorted population up to the maximum size
    return sorted_population[:POPULATION_SIZE]


def evaluate_fitness(population):
    fitnesses = []
    for solution in population:
        model = RandomForestClassifier()
        scores = cross_validate(model, X[:, solution], y, cv=5, scoring='f1')
        fitnesses.append(scores['test_score'].mean())
    return fitnesses


def optimize_feature_subset(X, y):
    # Initialize population
    population = initialize_population(X.shape[1], POPULATION_SIZE)
    fitnesses = evaluate_fitness(population)
    all_fitnesses = [fitnesses] # store fitness values in all generations

    previous_max_fitness = -float("inf") # minimum possible fitness value
    no_progress_count = 0 # counter for number of generations with no progress

    for generation in tqdm(range(MAX_GENERATIONS)):
        #print(f"Generation {generation}: population size = {len(population)}, fitnesses size = {len(fitnesses)}")
        #print(f"Population: {population}")
        #print(f"Fitnesses: {fitnesses}")

        # Select fittest solutions for next generation
        population = select_fittest(population, fitnesses)

        # Generate offspring through crossover and mutation
        offspring = []
        for _ in range(len(population)//2):
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = two_point_crossover(parent1, parent2)
            child1 = mutate(child1, X.shape[1], 1/X.shape[1])
            child2 = mutate(child2, X.shape[1], 1/X.shape[1])
            offspring.extend([child1, child2])

        # Evaluate fitness of offspring
        offspring_fitnesses = evaluate_fitness(offspring)

        # Select fittest solutions for next generation
        population = select_fittest(list(population) + offspring, fitnesses + offspring_fitnesses)


        # Evaluate fitness of current population
        fitnesses = evaluate_fitness(population)
        all_fitnesses.append(fitnesses)

        # Check for progress
        current_max_fitness = max(fitnesses)
        if current_max_fitness <= previous_max_fitness:
            no_progress_count += 1
        else:
            no_progress_count = 0
            previous_max_fitness = current_max_fitness

        # Terminate if no progress for 5 generations
        if no_progress_count >= 5:
            break

    # Return fittest solution
    fittest_index = fitnesses.index(max(fitnesses))
    return population[fittest_index]

In [ ]:
# Define X and y
X = train_features
y = train_labels

selected_feature_indices = optimize_feature_subset(train_features, train_labels)

##############################################################################

selected_train_features = train_features[:, selected_feature_indices]
selected_test_features = test_features[:, selected_feature_indices]

# Check the shape of the feature arrays
print("Shape of train_features:", selected_train_features.shape)
print("Shape of test_features:", selected_test_features.shape)

###############################################################################

In [ ]:
ga_scores = acc_score(selected_train_features, selected_test_features, train_labels, test_labels)

In [ ]:
# Sorting by column 'Classifier'
ga_sorted_scores = ga_scores.sort_values(by=['F1_Score'], ascending=False)
print("Results on test data")
print("-----------------------------")

ga_sorted_scores.head(10)

In [ ]:
ga_sorted_scores.to_csv("Classification with Ensemble Model After Applying GA.csv")